In [ ]:
import pandas as pd
import numpy as np

def compare_dataframes(df1, df2, key_columns):
    """
    对比两个DataFrame，计算所有非键列的增长比
    参数:
        df1: 时段1的DataFrame
        df2: 时段2的DataFrame
        key_columns: 用于匹配的键列列表
    
    返回:
        包含增长比结果的合并DataFrame
    """
    # 合并数据
    merged = pd.merge(df1, df2, on=key_columns, suffixes=('_时段1', '_时段2'), how='inner')
    
    # 找出所有数值型列（自动识别）
    numeric_cols = df1.select_dtypes(include=np.number).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in key_columns]
    
    # 计算数值列的增长比
    for col in numeric_cols:
        col1 = f'{col}_时段1'
        col2 = f'{col}_时段2'
        
        # 计算增长比
        merged[f'{col}_增长比'] = np.where(
            merged[col1] != 0,
            (merged[col2] - merged[col1]) / merged[col1],
            np.nan
        )
        
        # 格式化
        merged[f'{col}_增长比'] = merged[f'{col}_增长比'].apply(
            lambda x: f"{x:.2%}" if pd.notnull(x) else 'N/A')
    
    # 对于非数值列，可以添加其他对比逻辑
    # 例如文本列的对比、分类列的对比等
    
    return merged

In [3]:
# 使用示例
df1 = pd.read_excel('C:\Project\zlwl_pure_backend\Calculator\status\产品表现父ASIN（2025-07-16~2025-07-16，全部广告）-804284494327939072.xlsx')
df2 = pd.read_excel('C:\Project\zlwl_pure_backend\Calculator\status\产品表现父ASIN（2025-07-17~2025-07-17，全部广告）-804283898024435712.xlsx')
key_columns = ['父ASIN', '店铺']

result_df = compare_dataframes(df1, df2, key_columns)
result_df.to_excel('全列增长对比结果.xlsx', index=False)